# 6º Período - Pré-processamento, Representação Vetorial e Classificação de Textos

In [ ]:
!pip install sklearn2pmml

In [33]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from pathlib import Path  
import glob
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn.pipeline import Pipeline 
from sklearn.svm import SVC
#import contractions

In [2]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Dados

In [3]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

dataset_path = '/content/gdrive/MyDrive/Musical_instruments_reviews.csv'

Mounted at /content/gdrive


In [4]:
df = pd.read_csv(dataset_path)

df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [5]:
df.shape

(10261, 9)

In [6]:
df.isna().sum()

reviewerID         0
asin               0
reviewerName      27
helpful            0
reviewText         7
overall            0
summary            0
unixReviewTime     0
reviewTime         0
dtype: int64

Como o número de NAs é insignificante, não há problema em ignorar essas linhas

In [7]:
df = df.dropna()
df.isna().sum()

reviewerID        0
asin              0
reviewerName      0
helpful           0
reviewText        0
overall           0
summary           0
unixReviewTime    0
reviewTime        0
dtype: int64

In [8]:
df.shape

(10227, 9)

## Função de pré-processamento dos dados usando RegEx e NLTK;

(lowercasing, remover pontuações, stopwords, lemmatize, etc);

In [9]:
def pre_processing_regex(texto, patterns):
  novo = texto.tolist()
  for frase in novo:
    novo_texto = re.sub(patterns, lambda m: m.group(0).lower(), frase)
    novo_texto = novo_texto.translate(str.maketrans('', '', string.punctuation))
    novo.append(novo_texto)
  return novo

#phrase = ['TESTANDO, 1. 2. 3. 123!!','Que vontade DE UM STROGONOFF CARA!!! SERIO BRO??? ;-;']

phrase = df['reviewerName'].head(1)
padrao= '[A-Z]+'

# pre_processing_regex(phrase,padrao)

Apesar de criar a função e ela estar funcional, não a utilziaremos por existirem diversos métodos mais eficientes e intuitivos para pré processar os dados, como utilizando o NLTK:

In [10]:
data = df[['reviewText','summary']]

A **primeira ação** ao texto é expandir as contrações das palavras, ou seja, "I´ve" se torna "I have" e são tokenizados juntos, ou seja, não são alocadas diferentes como em "I" "have". A **segunda ação** é a Tokenização, ou seja, cada palavra individual vai ser dividida em um token. A **terceira ação** é converter todas as palavras para a letra minúscula. A **quarta ação** é para remover as pontuações. A **quinta ação** é para remover as chamadas "stop words", também chamadas de palavras vazias, ou seja, palavras sem significado (já definidas em uma função). A **sexta e última ação** é a chamada de "Lemmatization", que classifica as palavras em adjetivo, verbo, pronome, substantivo e etc.

In [11]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


def pre_processing_nltk(data):
  for column in data.columns:
 #   data[column] = data[column].apply(lambda x: [contractions.fix(word) for word in x.split()])

    data[column] = data[column].apply(word_tokenize)

    data[column] = data[column].apply(lambda x: [word.lower() for word in x])
 
    data[column] = data[column].apply(lambda x: [word for word in x if word not in string.punctuation])

    data[column] = data[column].apply(lambda x: [word for word in x if word not in stop_words])

    data[column] = data[column].apply(nltk.tag.pos_tag)

    data[column] = data[column].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

    wnl = WordNetLemmatizer()
    data[column] = data[column].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])

  return data

def preprocessnltk(data):
  data['reviewText'] = data['reviewText'].apply(word_tokenize)

  data['reviewText'] = data['reviewText'].apply(lambda x: [word.lower() for word in x])
 
  data['reviewText'] = data['reviewText'].apply(lambda x: [word for word in x if word not in string.punctuation])

  data['reviewText'] = data['reviewText'].apply(lambda x: [word for word in x if word not in stop_words])

  data['reviewText'] = data['reviewText'].apply(nltk.tag.pos_tag)

  data['reviewText'] = data['reviewText'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

  wnl = WordNetLemmatizer()
  data['reviewText'] = data['reviewText'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])

  return data

#data = pre_processing_nltk(data)

In [ ]:
data = pre_processing_nltk(data)

Agora, vamos retirar os dados do formato lista para que possamos utilizar normalmente os seguintes passos:

In [ ]:
data['reviewText'] = [' '.join(map(str,l)) for l in data['reviewText']]
data['summary'] = [' '.join(map(str,l)) for l in data['summary']]

In [17]:
data.head()

,reviewText,summary
0,much write exactly 's suppose filter pop sound...,good
1,product exactly quite affordable.i realize dou...,jake
2,primary job device block breath would otherwis...,job well
3,nice windscreen protects mxl mic prevents pops...,good windscreen money
4,pop filter great look performs like studio fil...,pop record vocal


## Tokenizando os textos com os métodos de BOW e TFIFD (sklearn);

Esse é o código a se utilizar caso, após o pré-processamento, o texto de cada linha esteja dentro de uma lista. Porque o BOW e TFIFD não entende quando os textos vem dentro de listas 

In [ ]:
# TFIFD

def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)  

In [ ]:
tfidf.fit(data['reviewText'])
tfidf.vocabulary_

In [ ]:
# BOW

vect = CountVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)

vect.fit(data['reviewText'])

In [ ]:
vect.vocabulary_

Agora, vamos utilizar BOW  e TFIFD com os textos de cada observação do dataset fora de uma lista:

### Bag Of Words

In [18]:
# BOW

vect = CountVectorizer()
vect.fit(data['reviewText'])

print(f'Tamanho: {len(vect.vocabulary_)}')
print(f'Contagem: {vect.vocabulary_}')

Tamanho: 17915
Contagem: {'much': 10363, 'write': 17733, 'exactly': 5909, 'suppose': 15288, 'filter': 6315, 'pop': 11948, 'sound': 14533, 'recording': 12810, 'crisp': 4221, 'one': 10973, 'low': 9472, 'price': 12162, 'amazon': 1443, 'might': 10020, 'well': 17406, 'buy': 2934, 'honestly': 7822, 'work': 17665, 'despite': 4708, 'product': 12242, 'quite': 12554, 'affordable': 1248, 'realize': 12694, 'double': 5139, 'screen': 13703, 'arrive': 1741, 'even': 5870, 'expected': 6005, 'as': 1768, 'add': 1144, 'bonus': 2594, 'carry': 3128, 'small': 14326, 'hint': 7748, 'smell': 14341, 'old': 10954, 'grape': 7205, 'candy': 3053, 'use': 16878, 'reminiscent': 12987, 'sake': 13526, 'stop': 14975, 'put': 12458, 'next': 10639, 'nose': 10744, 'record': 12804, 'dif': 4805, 'need': 10567, 'expensive': 6013, 'may': 9799, 'come': 3675, 'please': 11831, 'aroma': 1726, 'like': 9260, 'mine': 10058, 'primary': 12180, 'job': 8734, 'device': 4757, 'block': 2510, 'breath': 2728, 'would': 17708, 'otherwise': 11112, 

Agora, para que possamos levar e treinar o modelo para prever essas palavras, codificaremos cada palavra para matriz. "Transform documents to document-term matrix."

In [19]:
bow = vect.transform(data['reviewText'])

#print(bow)

print(bow.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [20]:
vect.get_feature_names_out()

array(['00', '000', '000s', ..., 'zylgian', 'zz', 'zzounds'], dtype=object)

In [21]:
vect.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

### TFIFD

In [27]:
# TFIDF

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(data['reviewText']) # equivalente a fazer vectorizer.fit() e depois vectorizer.transform()

In [28]:
vectorizer.get_feature_names_out()

array(['00', '000', '000s', ..., 'zylgian', 'zz', 'zzounds'], dtype=object)

In [29]:
vectorizer.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': None,
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

In [30]:
print(tfidf.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Utilizando classificadores clássicos de Machine Learning;


### Utilizando Support Vector Machine (SVM):

In [38]:
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split

In [41]:
x = tfidf
y = df['overall']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [42]:
clf = svm.SVC()
clf.fit(x_train,y_train)

SVC()

In [44]:
print(f'A acurácia do algoritmo de Support Vector Machine foi de {round(clf.score(x_test,y_test)*100,2)}%')

A acurácia do algoritmo de Support Vector Machine foi de 68.18%


Agora, vamos ver se existe alguma diferença utilizando BOW ao invés de TFIDF:

In [45]:
x = bow
y = df['overall']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [46]:
clf = svm.SVC()
clf.fit(x_train,y_train)

SVC()

In [47]:
print(f'A acurácia do algoritmo de Support Vector Machine foi de {round(clf.score(x_test,y_test)*100,2)}%')

A acurácia do algoritmo de Support Vector Machine foi de 67.74%


Analisa-se que não há diferença significativa entre os dois métodos de classificação, podendo-se utilizar qualquer um

### Naive Bayes:

In [48]:
from sklearn.naive_bayes import GaussianNB

In [54]:
dense_tfidf = tfidf.todense()

In [55]:
x = dense_tfidf
y = df['overall']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [56]:
gnb = GaussianNB()

In [57]:
gnb.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


GaussianNB()

In [60]:
y_pred = gnb.predict(x_test)

print("Number of mislabeled points out of a total %d points : %d" % (x_test.shape[0], (y_test != y_pred).sum()))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


Number of mislabeled points out of a total 2046 points : 1325


In [64]:
print(f'Vimos que a acurácia não foi das melhores, atingindo somente um percentual de {round((1-1325/2046)*100,2)}%')

Vimos que a acurácia não foi das melhores, atingindo somente um percentual de 35.24%


## Tokenização dos textos usando o Tensorflow (Tokenizer e método text_to_sequences);

## Rede Neural Recorrente Simples com camada de Embedding;

## Substituindo a RNN simples por uma LSTM unidirecional e uma bidimensional;